This file provides a compilation of different analyses performed on the Grapevine Model. Some code might not work straight away. Direct any questions to s.zhydkov@warwick.ac.uk. 

## Setup and Functions

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

using DataFrames, Statistics, LaTeXStrings, GraphIO, GraphPlot, Graphs, PyPlot, SNAPDatasets, StatsBase, JLD2

using PyPlot: grid as grd
color_list = ["#7fc97f","#beaed4","#fdc086","#ffff99","#386cb0"]
rc("font", size=14)
rc("mathtext", fontset="cm")
rc("legend", title_fontsize=12)

using Revise

includet("GrapevineModel.jl")
using .GrapevineModel

### Load and clean data

In [ ]:
file = "mega_data"
all_data = load_object("data/"*file*".jld2")

In [ ]:
all_data |> keys |> first

In [ ]:
col_names = [:mu10, :mu01, :p, :theta, :sources, :greedy_set, :greedy, :opt_set, :opt]
col_types = [fill(Float64, 4); Vector{Int64}; Set{Int}; Float64; Set{Int}; Float64]
log_df = DataFrame(col_names .=> [type[] for type in col_types])

## DIfference between greedy and optimal

In [ ]:
# log_df = DataFrame(params = NTuple{5, Any}[], greedy_set = Set{}[], greedy = Float64[], 
#     optimal_set = Set{}[], optimal = Float64[])

for x in keys(all_data)
#     println("Key: ", x)
    data = all_data[x]
    new_data = Dict()
    for key in keys(data)
        new_data[Set(key)] = data[key]
    end
    
    log = []
    
    if !check_greedy(new_data, 0.01, log)
        printstyled("Key: $x\n", color=:red)
        for l in log
            push!(log_df, [x..., l...])
        end
    end
    
#     println()
end

In [ ]:
log_df.diff = (log_df.opt .- log_df.greedy) ./ log_df.greedy

In [ ]:
using CSV
CSV.write("data/greedy_opt_diff_v2.csv", log_df)

## Testing additivity of influence

In [ ]:
for k in keys(all_data)
    println(k)
end

In [ ]:
key = (0.03970682084881814, 0.09844508319909746, 0.976945252809658, 0.7800567754188759)
data0 = all_data[key] #choose one experiment

data = Dict()
for key in keys(data0)
    data[Set(key)] = data0[key]
end
data[Set{Int64}([])] = 0

In [ ]:
key

In [ ]:
all_sets = sort(collect(keys(data)), by = x -> length(x))
sources = all_sets[end]
m = length(sources)

In [ ]:
marginals = Dict()
for s in sources
    s_marg = Dict()
    for i in 1:m
        sets_with_s = [set for set in all_sets if length(set) == i && (s in set)]
        s_marg[i] = [(data[set] - data[setdiff(set, [s])]) for set in sets_with_s]
    end
    marginals[s] = s_marg
end

In [ ]:
t = 1:m
s = rand(sources)
for i in t
    values = marginals[s][i]
    scatter(i*ones(Int, length(values)), values)
end

In [ ]:
for (A, B) in Iterators.product(all_sets, all_sets)
#     println(A, B, "\r")
    if data[A] + data[B] > 1.1*(data[union(A, B)] + data[intersect(A, B)])
        println("$A and $B violate supermodularity by
            $(data[A] + data[B] - (data[union(A, B)] + data[intersect(A, B)]))")
    end
end

In [ ]:
A = Set([789, 302, 882, 780, 2243])
B = Set([882, 541, 2243, 3416])
println(data[A])
println(data[B])
println(data[union(A, B)])
println(data[intersect(A, B)])

In [ ]:
modularity_data = ones(m+1, m+1)
count_data = zeros(m+1, m+1)
for (A, B) in Iterators.product(all_sets, all_sets)
    if isempty(intersect(A, B)) #disjoint sets
        
        a = data[union(A, B)] - (data[A] + data[B]) # excess gain (or loss) when putting 2 sets together
        if a < modularity_data[length(A)+1, length(B)+1]
            modularity_data[length(A)+1, length(B)+1] = copy(a)
        end
#         count_data[length(A)+1, length(B)+1] += 1
    end
end
# modularity_data = modularity_data ./ count_data
map!(x -> x == 1 ? 0 : x, modularity_data, modularity_data)

In [ ]:
imshow(modularity_data)
colorbar()

In [ ]:
findmin(modularity_data)

### Estimating the curvature

The total curvature of a set function $f$ is defined as $c = 1 - \min_{X, j \notin S} \frac{f(X) - f(X-j)}{f(\{j\})}$. Essentially, this measures how small the marginal contribution of any element to any set can get. If $f$ is increasing and submodular, $c \in [0, 1]$, where $c=0$ means that $f$ is additive.

In [ ]:
min_max = [1.0, 0.0]
for s in sources
    single_marg = marginals[s][1][1]
    all_marg = vcat([marginals[s][i] for i in 2:m]...)
    if minimum(all_marg)/single_marg < min_max[1]
        min_max[1] = minimum(all_marg)/single_marg
    end
    if maximum(all_marg)/single_marg > min_max[2]
        min_max[2] = maximum(all_marg)/single_marg
    end
end

In [ ]:
min_max

### Generalised Curvature and Submodularity Ratio

Estimating the parameters from the Bian et al. paper.

In [ ]:
all_data |> first |> last

In [ ]:
function gen_curvature(data)
    new_data = Dict()
    for key in keys(data)
        new_data[Set(key)] = data[key]
    end
    new_data[Set{Int64}([])] = 0
    
    all_sets = sort(collect(keys(new_data)), by = x -> length(x))
    
    cmin = 1.0
    cmax = 0.0
    
    for (S, O) in Iterators.product(all_sets, all_sets)
        for i in setdiff(S, O)
            ratio = 1-(new_data[union(S, O)] - new_data[union(setdiff(S, Set([i])), O)])/(new_data[S] - new_data[setdiff(S, Set([i]))])
            if ratio < cmin
                cmin = ratio
            elseif ratio > cmax
                cmax = ratio
            end
        end
    end
    return cmin, cmax
end

In [ ]:
 gen_curvature(collect(values(all_data))[100])

In [ ]:
function submod_ratio(data)
    new_data = Dict()
    for key in keys(data)
        new_data[Set(key)] = data[key]
    end
    new_data[Set{Int64}([])] = 0
    
    all_sets = sort(collect(keys(new_data)), by = x -> length(x))
    
    cmin = 1.0
    cmax = 0.0
    
    for (S, O) in Iterators.product(all_sets, all_sets)
        if isempty(setdiff(O, S))
            continue
        end
        omega_marg = new_data[union(S, O)] - new_data[S]
        indiv_marg = sum([(new_data[union(S, Set([i]))] - new_data[S]) for i in setdiff(O, S)])

        ratio = indiv_marg/omega_marg
        if ratio < cmin
            cmin = ratio
        elseif ratio > cmax
            cmax = ratio
        end
    end
    return cmin, cmax
end

In [ ]:
submod_ratio(collect(values(all_data))[100])

In [ ]:
curvatures = []
submods = []

for i in 1:length(all_data)
    data = collect(values(all_data))[i]
    sources = sort(collect(keys(data)), by = length)[end]
    if data[sources] < 0.05
        continue
    end

    push!(curvatures, gen_curvature(data)[2])
    push!(submods, submod_ratio(data)[1])
end

In [ ]:
plot(sort(submods))

In [ ]:
plot(sort(1 .- exp.(-curvatures)))

In [ ]:
approx = @. (1/curvatures) * (1 - exp(-curvatures * submods))

In [ ]:
plot(sort(approx))

In [ ]:
findall(submods .< 0)

In [ ]:
data = all_data[collect(keys(all_data))[141]]

new_data = Dict()
for key in keys(data)
    new_data[Set(key)] = data[key]
end

all_sets = sort(collect(keys(new_data)), by = length)
for set in all_sets
    println(set, ": ", new_data[set])
end

In [ ]:
sort(curvatures, rev=true)

## Posterior as a function of the message

In [ ]:
g = loadsnap(:facebook_combined) # our favourite network

# fix the params
n = nv(g)

theta = 0.7# prior probability the true state is 1
p = 1 # dropout rate
mu10 = 0.1 # mutation from 1 to 0
mu01 = 0.15; # mutation from 0 to 1

In [ ]:
figure(figsize=(8,6))

m = 20
for d in 2:2:10
    levels = d*ones(Int, m)
    messages = [[ones(Int, i); zeros(Int, m-i)] for i in 0:m]
    values = GrapevineModel.posterior.(theta, messages, Ref(levels), mu10, mu01)
    plot(0:m, values, label="depth = $d")
end

axhline(theta, ls="--", c="0.5", label="prior")

xticks(0:m)
grd(ls=":")
legend()
xlabel("Number of 1's received")
ylabel("Learned posterior")

savefig("figs/posterior_comp$m.pdf")

### Learned posterior vs number of corrupted sources

In [ ]:
# g = loadsnap(:facebook_combined) # our favourite network

# fix the params
n = nv(g)

theta = 0.7# prior probability the true state is 1
p = 0.9 # dropout rate
mu10 = 0.1 # mutation from 1 to 0
mu01 = 0.15; # mutation from 0 to 1

In [ ]:
g = path_graph(1)

In [ ]:
m = 20
sources = collect(1:m)
original_msg = Dict(sources .=> ones(Int, length(sources)))
instance = Grapevine(g, sources, original_msg, mu10, mu01, p, theta)
levels = [[ones(Int, m÷2)*3; ones(Int, m÷2)*5]]

In [ ]:
data = zeros(m÷2+1, m÷2+1)
for i in 0:m÷2, j in 0:m÷2
    original_msg = Dict(sources .=> [zeros(Int, i); ones(Int, m÷2-i); zeros(Int, j); ones(Int, m÷2-j)])
    instance = Grapevine(g, sources, original_msg, mu10, mu01, p, theta)
    data[i+1, j+1] = mean([x[1] for x in values(run_experiment(instance, 2000, levels_all=levels))])
end

In [ ]:
imshow(data)
colorbar()
ylabel("#corrupted sources at dist 3")
xlabel("#corrupted sources at dist 5")

savefig("figs/posterior_corr_heatmap.pdf")

### Learned posterior vs the number of corrupted sources in a realistic setting

In [ ]:
g = loadsnap(:facebook_combined) # our favourite network

# fix the params
n = nv(g)

theta = 0.7# prior probability the true state is 1
p = 0.9 # dropout rate
mu10 = 0.1 # mutation from 1 to 0
mu01 = 0.15; # mutation from 0 to 1

m = 20
sources = sample(vertices(g), m; replace=false)
original_msg = Dict(sources .=> ones(Int, length(sources)))
instance = Grapevine(g, sources, original_msg, mu10, mu01, p, theta)

In [ ]:
V_sample = sort(vertices(g), by = v -> mean(gdistances(g, v)[sources]))[100:500:end] #good spread of distances
# V_sample = sample(vertices(g), 10; replace=false) #random sample

In [ ]:
for v in V_sample
    println(mean(gdistances(g,v)))
end

In [ ]:
data = Dict()

for i in 1:m-1
    avg = zeros(length(V_sample))
    for j in 1:5
        S_corr = sample(sources, i; replace=false)
        original_msg = Dict(sources .=> [s in S_corr ? 0 : 1 for s in sources])
        instance = Grapevine(g, sources, original_msg, mu10, mu01, p, theta)
        dataс = run_experiment(instance, 100)
        avg += eachrow(hcat(values(dataс)...)'[:, V_sample]) |> mean
    end
    data[i] = avg/5
end

In [ ]:
plot_data = Dict()
for i in 1:length(V_sample)
    plot_data[V_sample[i]] = [data[j][i] for j in 1:m-1]
end
# delete!(plot_data, 701)

In [ ]:
for v in sort(keys(plot_data) |> collect, by = v -> mean(gdistances(g, v)[sources]))
    plot(1:m-1, plot_data[v], label="$(mean(gdistances(g, v)[sources]))", alpha=0.5, lw=1.5)
end
grd(ls=":")
xticks(1:2:19)
legend(title="avg dist \nto sources", bbox_to_anchor=(1, 1))
xlabel("Number of Corrupted Sources")
ylabel("Learned Posterior")

savefig("figs/posterior_vs_corrupted.pdf", bbox_inches="tight")

## Checking greedy on small graphs

In [ ]:
file = "small_graph_analytic"
all_data = load_object("data/"*file*".jld2")

In [ ]:
for graph in keys(graphs)
    println(graph, ": ", length([key for key in keys(all_data) if key[1] == graph]))
end

In [ ]:
log_df = DataFrame(graph = String[], params = NTuple{5, Any}[], greedy_set = Set{}[], greedy = Float64[], 
    optimal_set = Set{}[], optimal = Float64[])

sus_keys = []
for x in keys(all_data)
#     println("Key: ", x)
    data = all_data[x]
#     if typeof(x[1]) != Float64
#         x = x[2:end]
#     end
    new_data = Dict()
    for key in keys(data)
        new_data[Set(key)] = data[key]
    end
    
    log = []
    
    if !check_greedy(new_data, 0.00, log)
        printstyled("Key: $x\n", color=:red)
        for l in log
            push!(log_df, [x[1], x[2:end], l...])
        end
        push!(sus_keys, x)
    end
    
#     println()
end

In [ ]:
using DataStructures
counter([key[1] for key in sus_keys])

In [ ]:
using GraphIO
graphs = loadgraphs("graphs/graph5c.g6", Graph6Format())
g = graphs["graph15"]

In [ ]:
using GraphPlot, Colors, Compose, Cairo
gplot(g)
# gplot(g, nodefillc = ["gray", "blue", "green", "red", "gray"], nodelabel = vertices(g))
# Compose.draw(PNG("figs/greedy_ce.png", 16cm, 16cm), gplot(g, nodefillc = ["gray", "red", "green", "gray", "blue"]))

In [ ]:
i = 11
key = sus_keys[i]
# key = ("graph5", 0.02542531593846606, 0.002130935243685572, 0.9811953948099317, 0.9132144838651844, [1, 3, 4])
println(key)
g_name = key[1]
params = key[2:5]
sources = key[end]

data = all_data[key]
new_data = Dict()
for key in keys(data)
    new_data[Set(key)] = data[key]
end

In [ ]:
all_data = run_influence_experiment(g=g, params=params, sources=sources, method=:analytic, 
    graph_name=g_name, data_path="data/greedy_ce.jld2")

In [ ]:
data = first(values(all_data))

new_data = Dict()
for key in keys(data)
    new_data[Set(key)] = data[key]
end

In [ ]:
all_sets = sort(collect(keys(new_data)), by = length)
for set in all_sets
    println(set, ": ", new_data[set])
end

In [ ]:
check_greedy(new_data, 0.02)

In [ ]:
g = graphs[g_name]
gp = gplot(g, nodefillc = ["gray", "blue", "green", "red", "gray"], nodelabel = vertices(g))
# Compose.draw(PNG("figs/greedy_ce.png", 16cm, 16cm), gp)

In [ ]:
Grapevine(g, sources, Dict(sources .=> ones(length(sources))), params...)

## Greedy with Centrality Heuristics

In [ ]:
g = loadsnap(:facebook_combined)

In [ ]:
g = loadgraph("graphs/ER_graph.lg")

In [ ]:
harmonic_centrality(g) = [sum(1 ./ gdistances(g,i)[gdistances(g, i) .> 0]) for i in vertices(g)]

In [ ]:
centralities = [closeness_centrality(g), pagerank(g), degree_centrality(g), harmonic_centrality(g), eigenvector_centrality(g)]

In [ ]:
col_names = [:mu10, :mu01, :p, :theta, :sources, :greedy_set, :greedy, :opt_set, :opt]
col_types = [fill(Float64, 4); Vector{Int64}; Set{Int}; Float64; Set{Int}; Float64]
log_df2 = DataFrame(col_names .=> [type[] for type in col_types])

In [ ]:
# log_df = DataFrame(params = NTuple{5, Any}[], greedy_set = Set{}[], greedy = Float64[], 
#     optimal_set = Set{}[], optimal = Float64[])

sus_keys = []
for x in keys(all_data)
#     println("Key: ", x)
    data = all_data[x]
#     if typeof(x[1]) != Float64
#         x = x[2:end]
#     end
    new_data = Dict()
    for key in keys(data)
        new_data[Set(key)] = data[key]
    end
    
    log = []
    
    if !check_greedy(new_data, 0.02, log, centralities)
        printstyled("Key: $x\n", color=:red)
        for l in log
#             println([x..., l...])
            push!(log_df2, [x..., l...])
        end
        push!(sus_keys, x)
    end
    
#     println()
end

In [ ]:
log_df = process_data(all_data, centralities...)

In [ ]:
filtered_df = log_df[(length.(log_df.sources)/2 .- 1) .< log_df.k .< (length.(log_df.sources)/2 .+ 1), :]

In [ ]:
algs_scores = [:avg, :greedy_score, :closeness_score, :pagerank_score, :degree_score, :harmonic_score]
plot_data = Dict()
for a in algs_scores
#     println(a)
    plot_data[a] = (filtered_df[!, a]-filtered_df.worst)./(filtered_df.opt-filtered_df.worst)
end

In [ ]:
using CSV
CSV.write("data/data_summary.csv", log_df)

In [ ]:
algs_scores = [:avg, :greedy_score, :closeness_score, :pagerank_score, :degree_score, :harmonic_score]
plot_data = Dict()
for a in algs_scores
#     println(a)
    plot_data[a] = (log_df[!, a]-log_df.worst)./(log_df.opt-log_df.worst)
end

In [ ]:
for a in algs_scores
    plot(sort(plot_data[a], rev=true), label=a)
end
legend()
PyPlot.grid(ls=":")
ylabel("Performance (normalised)")
xlabel("Number of instances, %")
yticks(0:0.1:1, ["Worst", collect(0.1:0.1:0.9)..., "Optimal"])
xticks(0:length(plot_data[:avg])/10:length(plot_data[:avg]), 0:10:100)
xlim([0, length(first(values(plot_data)))])
savefig("figs/facebook_exp.pdf", bbox_inches="tight")
savefig("figs/facebook_exp.png", bbox_inches="tight")


In [ ]:
g

In [ ]:
plot_data[:closeness_score] .== plot_data[:radiality_score]

In [ ]:
plot_data[:radiality_score]

### Influence Exp on E-R

In [ ]:
file = "influence_exp_ER"
all_data = load_object("data/"*file*".jld2")

In [ ]:
g = loadgraph("graphs/ER_graph.lg")
centralities = [closeness_centrality(g), pagerank(g), degree_centrality(g), harmonic_centrality(g), eigenvector_centrality(g)]

In [ ]:
log_df = process_data(all_data, centralities...)

In [ ]:
algs_scores = [:avg, :greedy_score, :closeness_score, :pagerank_score, :degree_score, :harmonic_score, :eigenvector_score]
plot_data = Dict()
for a in algs_scores
#     println(a)
    plot_data[a] = (log_df[!, a]-log_df.worst)./(log_df.opt-log_df.worst)
end

In [ ]:
for a in algs_scores
    plot(sort(plot_data[a], rev=true), label=a)
end
legend()
PyPlot.grid(ls=":")
ylabel("Performance (normalised)")
xlabel("Number of instances, %")
yticks(0:0.1:1, ["Worst", collect(0.1:0.1:0.9)..., "Optimal"])
xticks(0:length(plot_data[:avg])/10:length(plot_data[:avg]), 0:10:100)
xlim([0, length(first(values(plot_data)))])
savefig("figs/ER_inf_exp.pdf", bbox_inches="tight")
savefig("figs/ER_inf_exp.png", bbox_inches="tight")


### Influence Exp on W-S 

In [ ]:
file = "influence_exp_WS"
all_data = load_object("data/"*file*".jld2")

In [ ]:
g = loadgraph("graphs/WS_graph.lg")
centralities = [closeness_centrality(g), pagerank(g), degree_centrality(g), harmonic_centrality(g), eigenvector_centrality(g)]

In [ ]:
log_df = process_data(all_data, centralities...)

In [ ]:
algs_scores = [:avg, :greedy_score, :closeness_score, :pagerank_score, :degree_score, :harmonic_score, :eigenvector_score]
plot_data = Dict()
for a in algs_scores
#     println(a)
    plot_data[a] = (log_df[!, a]-log_df.worst)./(log_df.opt-log_df.worst)
end

In [ ]:
for a in algs_scores
    plot(sort(plot_data[a], rev=true), label=a)
end
legend()
PyPlot.grid(ls=":")
ylabel("Performance (normalised)")
xlabel("Number of instances, %")
yticks(0:0.1:1, ["Worst", collect(0.1:0.1:0.9)..., "Optimal"])
xticks(0:length(plot_data[:avg])/10:length(plot_data[:avg]), 0:10:100)
xlim([0, length(first(values(plot_data)))])
savefig("figs/WS_inf_exp.pdf", bbox_inches="tight")
savefig("figs/WS_inf_exp.png", bbox_inches="tight")


## Watts-Strogatz Experiment

In [ ]:
file = "greedy_exp2"
all_data_even = load_object("data/"*file*".jld2")
file = "greedy_exp2_odd"
all_data_odd = load_object("data/"*file*".jld2")

In [ ]:
all_data = merge(all_data_even, all_data_odd)

In [ ]:
t = 3:19
algs = [:greedy, :random, :closeness, :pagerank, :degree, :harmonic] 
data=[]

plot_data = Dict(algs .=> [[[] for _ in t] for a in algs])
for i in 1:length(t)
    k = t[i]
    data = [all_data[key] for key in keys(all_data) if key[end] == k]
    for d in data
        for a in algs
            push!(plot_data[a][i], (d[a]-d[:random])/d[:random])
        end
    end
end

In [ ]:
for a in algs
    if a == nothing
        errorbar(0:length(t)-1, mean.(plot_data[a]), label=a, yerr=std.(plot_data[a]))
    else
        errorbar(0:length(t)-1, mean.(plot_data[a]), label=a)
    end

end
legend()
xticks(0:length(t)-1, t)
xlabel("Budget, "*L"k")
ylabel("Performance over random")
PyPlot.grid(ls=":")
xlim([0, length(t)-1])
savefig("figs/WS_exp.png", bbox_inches="tight")
savefig("figs/WS_exp.pdf", bbox_inches="tight")